### Importing Libraries

In [1]:
import os
import datetime
import pandas as pd
from scipy import signal

### Setting Constants

In [2]:
ROOT_DIRECTORY = "../DATASETS/bci_competition_3/p300"

In [3]:
SEGMENTED_FILES = ROOT_DIRECTORY+"/segmented_files"

In [4]:
NUMBER_CHANNELS = 64
INTER_STIMULI_SIZE = 42
NUMBER_CHARACTERS = 85
NUMBER_TRIALS = 15
NUMBER_ROWS_COLUMNS = 12
NUMBER_STIMULI = NUMBER_TRIALS * NUMBER_ROWS_COLUMNS #180
SEGMENT_BEGIN = 0
SEGMENT_SIZE = 160
SIGNAL_FREQUENCY = 240
LOWCUT = 0.1
HIGHCUT = 10.0
FILTER_ORDER = 5
DECIMATION_FACTOR = 12

In [105]:
SUBJECTS = ['A','B']
SIGNAL_FILES = ["/subject_A/Subject_A_Train_Signal.txt","/subject_B/Subject_B_Train_Signal.txt"]
STIMULUS_TYPE_FILES = ["/subject_A/Subject_A_Train_StimulusType.txt","/subject_B/Subject_B_Train_StimulusType.txt"]
STIMULUS_CODE_FILES = ["/subject_A/Subject_A_Train_StimulusCode.txt","/subject_B/Subject_B_Train_StimulusCode.txt"]

### Defining Functions

In [106]:
def chebyshev1_filter(signals,lowcut, highcut, order,fs=240):

    lowcut = 2 * lowcut / fs
    highcut = 2 * highcut /fs   
  
    b, a = signal.cheby1(order, 0.1,[lowcut, highcut],'bandpass')
    
    filtered_signals = []
    for i in range(len(signals)):
        signal_decimated = signal.decimate(signals[i],DECIMATION_FACTOR)
        filtered_signals.append(signal.lfilter(b, a, signal_decimated))        

    return filtered_signals 

In [128]:
def extract_values(file_values):
    all_extracted_values = []
    for i_char in file_values:
        separated_values = []
        for i_stimuli in range(0,len(file_values[i_char]),INTER_STIMULI_SIZE):
            separated_values.append(file_values[i_char][i_stimuli])
            if(len(separated_values) == 180):
                break
        all_extracted_values.append(separated_values)
    return all_extracted_values

### Loading, Selecting Values and Saving Files

In [ ]:
begin_time = datetime.datetime.now()

if not os.path.exists(SEGMENTED_FILES):
    os.makedirs(SEGMENTED_FILES)

for sigs,stims,codes,subject in zip(SIGNAL_FILES,STIMULUS_TYPE_FILES,STIMULUS_CODE_FILES,SUBJECTS):
    sigs_file = pd.read_csv(ROOT_DIRECTORY+sigs, delim_whitespace=True, header=None)
    stims_file = pd.read_csv(ROOT_DIRECTORY+stims, delim_whitespace=True, header=None)
    codes_file = pd.read_csv(ROOT_DIRECTORY+codes, delim_whitespace=True, header=None)
    
    count_char = 0
    for temp_char in range(0,sigs_file.shape[0],NUMBER_CHANNELS):

        temp_signals = sigs_file.iloc[temp_char:(temp_char+NUMBER_CHANNELS)]

        # Segmenting signals
        segments = []
        for temp_index,temp_signal in temp_signals.iterrows():
            for i_begin in range(0,len(temp_signal),INTER_STIMULU_SIZE):
                segments.append(temp_signal[i_begin+SEGMENT_BEGIN:i_begin+SEGMENT_BEGIN+SEGMENT_SIZE])
                if(i_begin >= INTER_STIMULU_SIZE*(NUMBER_STIMULI-1)):
                    break

        # Filtering segmented signals
        sigs_seg_filtered = chebyshev1_filter(segments,LOWCUT, HIGHCUT, FILTER_ORDER,SIGNAL_FREQUENCY)

        # Checking the if file exist
        folder_to_save = SEGMENTED_FILES+'/'+subject
        if not os.path.exists(folder_to_save):
                os.makedirs(folder_to_save)

        pd.DataFrame(sigs_seg_filtered).to_csv(folder_to_save+'/'+subject+'_'+str(count_char)+'_char_filtered_signals.csv',index=False)
        count_char = count_char + 1
            

    # Extracting codes values
    codes_extracted = extract_values(codes_file)
    pd.DataFrame(codes_extracted).to_csv(folder_to_save+'/'+subject+'_char_codes.csv',index=False)
    
    # Extracting stimuli values
    stimuli_extracted = extract_values(stims_file)
    pd.DataFrame(stimuli_extracted).to_csv(folder_to_save+'/'+subject+'_char_stimuli.csv',index=False)

    
end_time = datetime.datetime.now()
print('Begin:')
print(begin_time)
print(end_time-begin_time)